# SI 618 WN Project Part I

## Project Title:
> Provide a descriptive working title for your project.

**Game Sales, Popularity, and Achievements: A Comparative Analysis Across Platforms**

## Team Members
> List each team member and include their uniqname

* Yufeng Song (yfsong)
* Ziqi Wang (Venchy)
* Muyu Lin (linmuyu)

## Overview
> Give a high level description of your project

Our project explores key trends in the gaming industry by analyzing data on game sales, player achievements, and platform preferences. We aim to understand how the same game performs across different platforms, how purchasing preferences vary across countries, and how game popularity evolves over time. Additionally, we will investigate the relationship between supported languages and sales, as well as the impact of pricing on game popularity across multiple platforms. By analyzing these factors, we hope to uncover insights into player behavior, market trends, and factors influencing game success.

## Motivation
> Explain why you chose this particular topic for your project.	Include the three "real-world" questions that you generated about the data, and be sure to explain what you hope to learn by answering them.

We chose this topic because gaming is a massive industry with a highly diverse audience, and understanding player preferences and market trends can provide valuable insights for developers, publishers, and gaming communities. Our project aims to answer the following real-world questions:

1. **How do in-game achievements compare across different platforms for the same game?**

  - By analyzing achievement data, we aim to understand whether players engage with a game differently depending on the platform they use. This could reveal differences in play styles, game difficulty adjustments, or platform-specific engagement trends.

2. **How do game purchase preferences vary by country and active playtime?**

  - We seek to determine whether purchasing behaviors differ based on regional preferences and player engagement levels. Understanding this can help developers tailor their marketing strategies and optimize game pricing for different audiences.

3. **How has the popularity of different game types changed over time?**

  - By tracking shifts in game genre popularity over time, we hope to uncover trends that indicate the rise and fall of specific genres. This insight could be useful for predicting future market demands and guiding game development strategies.

By answering these questions, we hope to gain a deeper understanding of the gaming landscape, helping stakeholders make data-driven decisions about game development, pricing, and distribution strategies.

## Data Sources
> List the two (or more) sources of data that you'll be using.  Provide URLs where appropriate.	**Explain how the two (or more) datasets complement each other.**

1. https://www.kaggle.com/datasets/artyomkruglov/gaming-profiles-2025-steam-playstation-xbox

2. https://github.com/Smipe-a/gamestatshub

- This Gaming Profiles Data from Kaggle include **game** and **player** data from three different **platforms**, PlayStation, Steam, and Xbox.
- For each platform, the **player** and **game** data is complemented by **achievement** data. The relationships between players, games, and achievements are one-to-many: each player can play multiple games, and each game can have multiple achievements.
- Data from the three platforms can be combined based on **game titles** to analyze shared games across platforms.

## Data Description
> List the variables of interest, the size of the data sets, missing values, etc.

### Common Datasets Across 3 Platforms & Variables of Interest:

#### Player Metadata
- **players.csv**: platform-specific `playerid` and `country` (Xbox data lacks country column)
- **purchased_games.csv** lists players' purchased games, with:
    - `playerid`
    - `libarary`: a list of games the player bought.
- **history.csv**: when the player unlocked the achievement.
    - `playerid`
    - `achievementid`
    - `date_acquired`

#### Game Metadata
- **games.csv**: game details such as `genres`, `developers`, `publishers`, `supported_language`, and `release_date`.
- **achievements.csv** maps achievements to their respective game, with:
    - `achievementid` combines the uniqe game id on the platform and the achievement id within the game
    - `gameid`
    - `title`
    - `description`
- **prices.csv**: games' price in various currencies, with `date_acquired` indicating the date when the price was recorded.

## Data Manipulation
> Mostly code in this section.  This is where you merge your data sets, as well as create new columns (if appropriate)

#### Merged Datasets Explained - Flattened Table For Raw Analysis
- Platform-Specific Datasets:
    - Player-Info Dataset: merged on `playerid`
    - Game-Info Dataset: merged on `gameid`
- Cross-Platform Game Dataset: merged on game `title` to combine shared game data from all three platforms.


**Notes**: duplicated gameid/playerid after merge due to the one-to-many relationship between players, games, and achievements. 

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## PS

In [192]:
import os

PS_DATA_FOLDER = "data/ps"

dfs = {}

for root, dirs, files in os.walk(PS_DATA_FOLDER):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            df_name = f"{os.path.basename(root)}_{file.replace('.csv', '')}".replace('.csv', '')  # Clean name
            dfs[df_name] = pd.read_csv(file_path)

for name, df in dfs.items():
    print(f"{name} - shape: {df.shape}")

ps_purchased_games - shape: (46582, 2)
ps_history - shape: (19510083, 3)
ps_prices - shape: (62816, 7)
ps_players - shape: (356600, 3)
ps_games - shape: (23151, 8)
ps_achievements - shape: (846563, 5)


In [ ]:
ps_achievements = dfs['ps_achievements']
ps_games = dfs['ps_games']
ps_history = dfs['ps_history']
ps_players = dfs['ps_players']
ps_prices = dfs['ps_prices']
ps_purchased_games = dfs['ps_purchased_games']

# check duplicates
print(ps_games.duplicated(subset=["gameid"]).sum())
print(ps_prices.duplicated(subset=["gameid"]).sum())
print(ps_achievements.duplicated(subset=["achievementid"]).sum())
print(ps_players.duplicated(subset=["playerid"]).sum())
print(ps_purchased_games.duplicated(subset=["playerid"]).sum())

0
32095
0
0
0


In [ ]:
ps_prices['gameid'].value_counts()

gameid
726666    3
723091    3
727730    3
727729    3
728792    3
         ..
421567    2
421164    2
421153    2
421368    2
423969    2
Name: count, Length: 30721, dtype: int64

In [196]:
ps_prices[ps_prices['gameid'] == 726666]

,gameid,usd,eur,gbp,jpy,rub,date_acquired
20167,726666,6.99,6.99,5.79,1100.0,NaN,2025-02-22
50835,726666,6.99,6.99,5.79,1100.0,NaN,2025-02-25
52307,726666,6.99,6.99,5.79,1100.0,NaN,2025-02-24


In [208]:
# for each gameid, keep only the most recent price entry
ps_prices = ps_prices.sort_values("date_acquired").drop_duplicates(subset=["gameid"], keep="last")
print(ps_prices.duplicated(subset=["gameid"]).sum())

0


### Player focus

In [213]:
ps_player_info = ps_players.merge(ps_history, on="playerid", how="left")

In [212]:
ps_player_info = ps_player_info.merge(ps_achievements, on="achievementid", how="left")

In [168]:
ps_player_info = ps_player_info.merge(ps_purchased_games, on="playerid", how="left")

In [169]:
ps_player_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19861683 entries, 0 to 19861682
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   playerid       int64  
 1   nickname       object 
 2   country        object 
 3   achievementid  object 
 4   date_acquired  object 
 5   gameid         float64
 6   title          object 
 7   description    object 
 8   rarity         object 
 9   library        object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.5+ GB


In [170]:
ps_player_info.sample(5)

,playerid,nickname,country,achievementid,date_acquired,gameid,title,description,rarity,library
10249753,1938676,petie170,Belgium,141933_2609420,2021-07-24 19:37:38,141933.0,Double Down,"Beat Tucker Morgan, the Offroad champ, in the...",Silver,"[756211, 682504, 755124, 406327, 618499, 55030..."
15586577,2676012,Mike_Opilot,United States,490129_4301266,2023-12-22 22:59:39,490129.0,Living Your Car Life,Started the first Menu Book.,Bronze,"[519939, 707901, 11019, 165840, 14603, 608466,..."
687077,299803,NewYorkUgly,Canada,9311_111077,2020-09-25 01:16:40,9311.0,A Blade of Memory,"Destroy all Teeth, Balloons and Jack-In-The-Bo...",Bronze,"[755124, 702257, 591550, 546924, 668727, 7690,..."
15375171,1937020,lirawapo,Mexico,670209_5436077,2024-04-26 11:52:17,670209.0,Abaddon,Defeated Abaddon.,Bronze,"[717169, 417808, 670209, 550305, 10939, 755124..."
13468353,315008,SpecimenLove,United Kingdom,479_5852,2011-08-09 20:03:22,479.0,Breadwinner,Earn the top score objective in any level,Silver,"[20571, 403080, 404720, 20793, 330726, 10423, ..."


### Game focus
date_acquired column is dropped as it indicates the timestamp when the price info was extracted from multiple databases and do not add helpful insights to our analysis.

In [214]:
ps_game_info = ps_games.merge(ps_achievements.rename(columns={'title': 'achievement_title'}), on="gameid", how="left")

In [215]:
ps_game_info = ps_game_info.merge(ps_prices, on="gameid", how="left")

In [216]:
ps_game_info = ps_game_info.drop(columns=['date_acquired'])

In [217]:
ps_game_info.rename(columns={'platform': 'PS_platform'}, inplace=True)

In [218]:
ps_game_info.rename(columns={'description': 'achievement_description'}, inplace=True)

In [219]:
ps_game_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626958 entries, 0 to 626957
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   gameid                   626958 non-null  int64  
 1   title                    626958 non-null  object 
 2   PS_platform              626958 non-null  object 
 3   developers               626416 non-null  object 
 4   publishers               626624 non-null  object 
 5   genres                   623044 non-null  object 
 6   supported_languages      319860 non-null  object 
 7   release_date             626958 non-null  object 
 8   achievementid            626958 non-null  object 
 9   achievement_title        626954 non-null  object 
 10  achievement_description  626942 non-null  object 
 11  rarity                   626958 non-null  object 
 12  usd                      535794 non-null  float64
 13  eur                      508924 non-null  float64
 14  gbp 

PS-only cols: rarity, PS_platform

In [220]:
ps_game_info.sample(5)

,gameid,title,PS_platform,developers,publishers,genres,supported_languages,release_date,achievementid,achievement_title,achievement_description,rarity,usd,eur,gbp,jpy,rub
77513,613296,Hidden Shapes: Cat Realm + Trick Or Cats,PS4,['YAW Studios'],['QUByte Interactive'],"['Puzzle', 'Collection']",NaN,2023-08-17,613296_5049459,HSTC - 01,Win 1st star in the puzzle - 01 and 02,Bronze,4.99,4.99,3.99,660.0,NaN
483073,557641,Quiz Thiz USA: Bronze Edition,PS5,['ThiGames'],['ThiGames'],['Educational & Trivia'],NaN,2022-11-26,557641_4665452,Answer 60 questions wrong,Answer 60 questions wrong,Bronze,2.49,2.49,1.99,330.0,NaN
446250,609590,Double Dragon Gaiden : Rise of the Dragons,PS4,['Secret Base'],"['Maximum Games', 'Modus Games', 'NiuGamer']","[""Beat 'em up""]","['Japanese', 'French', 'Spanish', 'German', 'I...",2023-07-27,609590_5025533,Beat'em Up,Special KO 3000 enemies across multiple sessions,Gold,24.99,24.99,19.99,NaN,1789.0
385550,695466,Monster Jam Showdown,PS4,['Milestone'],['Milestone'],"['Automobile', 'Arcade Racing']","['Japanese', 'French', 'German', 'Italian', 'C...",2024-08-29,695466_5616274,Loads of Checkmarks,Complete 15 secondary objectives.,Bronze,24.99,24.99,22.49,5830.0,3579.0
101117,573324,DreadOut 2,PS5,['Digital Happiness'],['Digerati'],['Survival Horror'],NaN,2022-07-20,573324_4768505,Night Rider,"Complete ""Ghost Rider""",Gold,13.99,19.99,15.99,3300.0,NaN


In [221]:
ps_game_info['achievementid'].value_counts()

achievementid
749375_6098396    1
650405_5312282    1
650406_5312373    1
650406_5312374    1
650406_5312375    1
                 ..
336965_3248824    1
336965_3248825    1
336965_3248826    1
336965_3248827    1
7593_93460        1
Name: count, Length: 626958, dtype: int64

## Steam

In [223]:
import os
import pandas as pd

STEAM_DATA_FOLDER = "data/steam"

dfs = {}

for root, dirs, files in os.walk(STEAM_DATA_FOLDER):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            df_name = f"{os.path.basename(root)}_{file.replace('.csv', '')}".replace('.csv', '')  # Clean name
            dfs[df_name] = pd.read_csv(file_path)

for name, df in dfs.items():
    print(f"{name} - shape: {df.shape}")


steam_purchased_games - shape: (102548, 2)
steam_reviews - shape: (1204534, 8)
steam_history - shape: (10693879, 3)
steam_friends - shape: (424683, 2)
steam_prices - shape: (4414273, 7)
steam_players - shape: (424683, 3)
steam_games - shape: (98248, 7)
steam_private_steamids - shape: (227963, 1)
steam_achievements - shape: (1939027, 4)


In [224]:
steam_achievements = dfs['steam_achievements']
steam_games = dfs['steam_games']
steam_history = dfs['steam_history']
steam_players = dfs['steam_players']
steam_prices = dfs['steam_prices']
steam_purchased_games = dfs['steam_purchased_games']

# check duplicates
print(steam_games.duplicated(subset=["gameid"]).sum())
print(steam_prices.duplicated(subset=["gameid"]).sum())
print(steam_achievements.duplicated(subset=["achievementid"]).sum())
print(steam_players.duplicated(subset=["playerid"]).sum())
print(steam_purchased_games.duplicated(subset=["playerid"]).sum())

0
4315808
0
0
0


In [225]:
# for each gameid, keep only the most recent price entry
steam_prices = steam_prices.sort_values("date_acquired").drop_duplicates(subset=["gameid"], keep="last")
print(steam_prices.duplicated(subset=["gameid"]).sum())

0


### Player focus

In [ ]:
st_player_info = steam_players.merge(steam_history, on="playerid", how="left")

In [ ]:
st_player_info = st_player_info.merge(steam_achievements, on="achievementid", how="left")

In [85]:
st_player_info = st_player_info.merge(steam_purchased_games, on="playerid", how="left")

# # Save player-focused data
# steam_player_info.to_csv('data/steam_players.csv', index=False)

In [86]:
st_player_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11113724 entries, 0 to 11113723
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   playerid       int64  
 1   country        object 
 2   created        object 
 3   achievementid  object 
 4   date_acquired  object 
 5   gameid         float64
 6   title          object 
 7   description    object 
 8   library        object 
dtypes: float64(1), int64(1), object(7)
memory usage: 763.1+ MB


In [87]:
st_player_info.shape

(11113724, 9)

In [88]:
st_player_info.head(5)

,playerid,country,created,achievementid,date_acquired,gameid,title,description,library
0,76561198287452552,Brazil,2016-03-02 06:14:20,NaN,NaN,NaN,NaN,NaN,"[10, 80, 100, 240, 2990, 6880, 6910, 6920, 698..."
1,76561198040436563,Israel,2011-04-10 17:10:06,NaN,NaN,NaN,NaN,NaN,"[10, 80, 100, 300, 20, 30, 40, 50, 60, 70, 130..."
2,76561198049686270,NaN,2011-09-28 21:43:59,NaN,NaN,NaN,NaN,NaN,NaN
3,76561198155814250,Kazakhstan,2014-09-24 19:52:47,NaN,NaN,NaN,NaN,NaN,NaN
4,76561198119605821,NaN,2013-12-26 00:25:50,NaN,NaN,NaN,NaN,NaN,"[47870, 108600, 550, 271590, 331470, 381210, 2..."


### Game focus

In [227]:
st_game_info = steam_games.merge(steam_achievements.rename(columns={'title': 'achievement_title'}), on="gameid", how="left")

In [228]:
st_game_info = st_game_info.merge(steam_prices, on="gameid", how="left")

In [229]:
st_game_info = st_game_info.drop(columns=['date_acquired'])

In [230]:
st_game_info.rename(columns={'description': 'achievement_description'}, inplace=True)

In [231]:
st_game_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1983659 entries, 0 to 1983658
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   gameid                   int64  
 1   title                    object 
 2   developers               object 
 3   publishers               object 
 4   genres                   object 
 5   supported_languages      object 
 6   release_date             object 
 7   achievementid            object 
 8   achievement_title        object 
 9   achievement_description  object 
 10  usd                      float64
 11  eur                      float64
 12  gbp                      float64
 13  jpy                      float64
 14  rub                      float64
dtypes: float64(5), int64(1), object(9)
memory usage: 227.0+ MB


In [232]:
st_game_info.shape

(1983659, 15)

In [234]:
st_game_info['achievementid'].value_counts()

achievementid
3266470_ACHIEVEMENT_5_MINUTES    1
2578790_Liquidator               1
2578790_Space_Rebel              1
2578790_Slugger                  1
2578790_Sergeant                 1
                                ..
1098080_ACHIEVEMENT_9            1
1098080_ACHIEVEMENT_8            1
1098080_ACHIEVEMENT_7            1
1098080_ACHIEVEMENT_6            1
1499240_ROMANROCKS               1
Name: count, Length: 1935882, dtype: int64

## XBOX

In [238]:
import os
import pandas as pd

XBOX_DATA_FOLDER = "data/xbox"

dfs = {}

for root, dirs, files in os.walk(XBOX_DATA_FOLDER):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            df_name = f"{os.path.basename(root)}_{file.replace('.csv', '')}".replace('.csv', '')
            dfs[df_name] = pd.read_csv(file_path)

for name, df in dfs.items():
    print(f"{name} - shape: {df.shape}")


xbox_purchased_games - shape: (46466, 2)
xbox_history - shape: (15275900, 3)
xbox_prices - shape: (22638, 7)
xbox_players - shape: (274450, 2)
xbox_games - shape: (10489, 7)
xbox_achievements - shape: (351111, 5)


In [239]:
xbox_achievements = dfs['xbox_achievements']
xbox_games = dfs['xbox_games']
xbox_history = dfs['xbox_history']
xbox_players = dfs['xbox_players']
xbox_prices = dfs['xbox_prices']
xbox_purchased_games = dfs['xbox_purchased_games']

# check duplicates
print(xbox_games.duplicated(subset=["gameid"]).sum())
print(xbox_prices.duplicated(subset=["gameid"]).sum())
print(xbox_achievements.duplicated(subset=["achievementid"]).sum())
print(xbox_players.duplicated(subset=["playerid"]).sum())
print(xbox_purchased_games.duplicated(subset=["playerid"]).sum())

0
11319
0
0
0


In [240]:
# for each gameid, keep only the most recent price entry
xbox_prices = xbox_prices.sort_values("date_acquired").drop_duplicates(subset=["gameid"], keep="last")
print(xbox_prices.duplicated(subset=["gameid"]).sum())

0


### Player focus

In [97]:
xb_player_info = xbox_players.merge(xbox_history, on="playerid", how="left")

In [98]:
xb_player_info = xb_player_info.merge(xbox_achievements, on="achievementid", how="left")

In [99]:
xb_player_info = xb_player_info.merge(xbox_purchased_games, on="playerid", how="left")

In [100]:
xb_player_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15545366 entries, 0 to 15545365
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   playerid       int64  
 1   nickname       object 
 2   achievementid  object 
 3   date_acquired  object 
 4   gameid         float64
 5   title          object 
 6   description    object 
 7   points         float64
 8   library        object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.0+ GB


In [101]:
xb_player_info.shape

(15545366, 9)

### Game focus

In [241]:
xb_game_info = xbox_games.merge(xbox_achievements.rename(columns={'title': 'achievement_title'}), on="gameid", how="left")

In [242]:
xb_game_info = xb_game_info.merge(xbox_prices, on="gameid", how="left")

In [243]:
xb_game_info = xb_game_info.drop(columns=['date_acquired', 'points'])

In [244]:
xb_game_info.rename(columns={'description': 'achievement_description'}, inplace=True)

In [245]:
xb_game_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323038 entries, 0 to 323037
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   gameid                   323038 non-null  int64  
 1   title                    323038 non-null  object 
 2   developers               304402 non-null  object 
 3   publishers               305059 non-null  object 
 4   genres                   304168 non-null  object 
 5   supported_languages      132805 non-null  object 
 6   release_date             323038 non-null  object 
 7   achievementid            323038 non-null  object 
 8   achievement_title        323037 non-null  object 
 9   achievement_description  322984 non-null  object 
 10  usd                      237247 non-null  float64
 11  eur                      226837 non-null  float64
 12  gbp                      234618 non-null  float64
 13  jpy                      0 non-null       float64
 14  rub 

## Game Info Across Platforms

In [246]:
len(ps_game_info['gameid'].unique())

23151

In [247]:
len(st_game_info['gameid'].unique())

98248

In [248]:
len(xb_game_info['gameid'].unique())

10489

In [249]:
ps_titles = set(ps_game_info['title'].str.lower().str.strip())
st_titles = set(st_game_info['title'].str.lower().str.strip())
xb_titles = set(xb_game_info['title'].str.lower().str.strip())

shared_ps_st = ps_titles.intersection(st_titles)
shared_ps_xb = ps_titles.intersection(xb_titles)
shared_st_xb = st_titles.intersection(xb_titles)
shared_all = ps_titles.intersection(st_titles, xb_titles)

print(f"Shared titles between PS & ST: {len(shared_ps_st)}")
print(f"Shared titles between PS & XB: {len(shared_ps_xb)}")
print(f"Shared titles between ST & XB: {len(shared_st_xb)}")
print(f"Shared across all three: {len(shared_all)}")


Shared titles between PS & ST: 5571
Shared titles between PS & XB: 5360
Shared titles between ST & XB: 5540
Shared across all three: 3815


In [250]:
def clean_game_info(df, platform):
    df = df.copy()
    
    # drop platform-specific columns (ps_game_info's 'rarity' and 'PS_platform')
    drop_cols = ['date_acquired', 'rarity', 'PS_platform']
    df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors='ignore')

    # add platform column
    df['platform'] = platform
    
    return df

In [251]:
ps_game_info_clean = clean_game_info(ps_game_info, 'ps')

In [252]:
ps_game_info_clean.columns

Index(['gameid', 'title', 'developers', 'publishers', 'genres',
       'supported_languages', 'release_date', 'achievementid',
       'achievement_title', 'achievement_description', 'usd', 'eur', 'gbp',
       'jpy', 'rub', 'platform'],
      dtype='object')

In [254]:
st_game_info_clean = clean_game_info(st_game_info, 'st')

In [255]:
st_game_info_clean.columns

Index(['gameid', 'title', 'developers', 'publishers', 'genres',
       'supported_languages', 'release_date', 'achievementid',
       'achievement_title', 'achievement_description', 'usd', 'eur', 'gbp',
       'jpy', 'rub', 'platform'],
      dtype='object')

In [256]:
xb_game_info_clean = clean_game_info(xb_game_info, 'xb')

In [257]:
xb_game_info_clean.columns

Index(['gameid', 'title', 'developers', 'publishers', 'genres',
       'supported_languages', 'release_date', 'achievementid',
       'achievement_title', 'achievement_description', 'usd', 'eur', 'gbp',
       'jpy', 'rub', 'platform'],
      dtype='object')

In [258]:
shared_games = set(ps_game_info_clean['title']) & set(st_game_info_clean['title']) & set(xb_game_info_clean['title'])

ps_game_info_clean = ps_game_info_clean[ps_game_info_clean['title'].isin(shared_games)]
st_game_info_clean = st_game_info_clean[st_game_info_clean['title'].isin(shared_games)]
xb_game_info_clean = xb_game_info_clean[xb_game_info_clean['title'].isin(shared_games)]

merged_game_info = pd.concat([ps_game_info_clean, st_game_info_clean, xb_game_info_clean], ignore_index=True)

print(f"Merged dataset shape: {merged_game_info.shape}")

Merged dataset shape: (431985, 16)


In [259]:
merged_game_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431985 entries, 0 to 431984
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   gameid                   431985 non-null  int64  
 1   title                    431985 non-null  object 
 2   developers               430744 non-null  object 
 3   publishers               430459 non-null  object 
 4   genres                   430507 non-null  object 
 5   supported_languages      274284 non-null  object 
 6   release_date             431985 non-null  object 
 7   achievementid            431807 non-null  object 
 8   achievement_title        431801 non-null  object 
 9   achievement_description  414137 non-null  object 
 10  usd                      398232 non-null  float64
 11  eur                      386835 non-null  float64
 12  gbp                      396497 non-null  float64
 13  jpy                      186153 non-null  float64
 14  rub 

## Long and Tidy Form
A tidy dataset follows the principles:
1. Each column is a single variable.
2. Each row is a single observation.
3. Each cell contains a single value.

#### Issues with the current merged_game_info:
- **We decide to leave it unchanged as our analysis will base on it**: The platform column is categorical, meaning the same game can appear multiple times (once per platform). This makes it long format.
- Price columns (usd, eur, etc.) are separate instead of melted into a single column, which makes the dataset wide instead of long, which is not tidy.

In [261]:
# convert price columns into a long format
tidy_game_info = merged_game_info.melt(
    id_vars=[
        "gameid", "title", "developers", "publishers", "genres",
        "supported_languages", "release_date", "achievementid",
        "achievement_title", "achievement_description", "platform"
    ],
    value_vars=["usd", "eur", "gbp", "jpy", "rub"],
    var_name="currency",
    value_name="price"
)

In [262]:
tidy_game_info.shape

(2159925, 13)

In [263]:
tidy_game_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2159925 entries, 0 to 2159924
Data columns (total 13 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   gameid                   int64  
 1   title                    object 
 2   developers               object 
 3   publishers               object 
 4   genres                   object 
 5   supported_languages      object 
 6   release_date             object 
 7   achievementid            object 
 8   achievement_title        object 
 9   achievement_description  object 
 10  platform                 object 
 11  currency                 object 
 12  price                    float64
dtypes: float64(1), int64(1), object(11)
memory usage: 214.2+ MB


In [264]:
tidy_game_info.sample(5)

,gameid,title,developers,publishers,genres,supported_languages,release_date,achievementid,achievement_title,achievement_description,platform,currency,price
895685,558846,Neon White,['Ben Esposito'],['Annapurna Interactive'],['Platformer'],"['Japanese', 'French', 'Spanish', 'German', 'I...",2022-12-13,558846_4677683,Mission 3 Complete,Complete Mission 3,ps,gbp,19.99
536466,11298,Rebel Galaxy,['Double Damage Games'],['Double Damage Games'],['strategy'],"['Japanese', 'French', 'Spanish', 'Russian', '...",2016-01-05,11298_127630,Some Muscle,Hire a mercenary,ps,eur,19.99
686078,1021100,Aircraft Carrier Survival,"['Gambit Games Studio', 'Creative Storm Entert...",['CreativeForge Games'],"['Action', 'Indie', 'Strategy']","['English', 'French', 'German', 'Polish', 'Sim...",2022-04-20,1021100_NEW_ACHIEVEMENT_1_5,Warrant Officer,Finish the second chapter of the campaign.,st,eur,16.79
1181085,1545700,Guns N' Runs,['Statera Studio'],['Statera Studio'],"['Action', 'Adventure', 'Indie']","['English', 'French', 'Simplified Chinese', 'J...",2021-03-30,1545700_ACHIEVEMENT_32,Bunkers? That's my thing,Beat the game as Noah,st,gbp,5.89
415337,495457,Mokoko X,['NAISU'],['NAISU'],['Action'],NaN,2022-04-22,495457_4334253,Tuck the Duck,Tuck the Duck has been defeated,xb,usd,11.49


In [265]:
tidy_game_info['achievementid'].value_counts()

achievementid
749591_6100112             5
384010_PIECE_OF_CAKE       5
384180_ACH_SHOP_A_HOLIC    5
384180_ACH_BLACKJACK       5
384180_ACH_BILLIONAIRE     5
                          ..
603576_4978957             5
603576_4978956             5
603576_4978955             5
603576_4978954             5
711118_5703719             5
Name: count, Length: 431807, dtype: int64

## Preprocessing

### 1. Check for missing values

In [124]:
# count missing values per column
null_counts = tidy_game_info.isnull().sum().reset_index()
null_counts.columns = ["col", "num_missing"]
null_counts["missing_pcnt"] = round((null_counts["num_missing"] / len(merged_game_info)) * 100, 2)

null_counts

,col,num_missing,missing_pcnt
0,gameid,0,0.00
1,title,0,0.00
2,developers,15850,0.27
3,publishers,79975,1.38
4,genres,24935,0.43
5,supported_languages,1588735,27.35
6,release_date,0,0.00
7,achievementid,40050,0.69
8,achievement_title,40325,0.69
9,achievement_description,4014060,69.11


#### Here’s how we decide to handle missing values effectively [Used GPT-4o for tailoring to table format]:

**Columns with Minor Missing Data (< 5%)**
| Column | Missing % | Suggested Action |
|---------|----------|-----------------|
| **developers** | 0.27% | Fill with "unknown" to maintain consistency. |
| **publishers** | 1.38% | Fill with "unknown" for completeness. |
| **genres** | 0.43% | Fill with "unknown", since games without a genre classification are rare. |
| **achievementid** | 0.69% | Likely an error or missing achievements, can be kept as NaN. |
| **achievement_title** | 0.69% | Likely corresponds to missing achievementid, can be kept as NaN. |

- **Action: Fill developers, publishers, and genres with "unknown", and leave missing achievements as NaN.**

---

**Columns with Moderate Missing Data (5% - 30%)**
| Column | Missing % | Suggested Action |
|---------|----------|-----------------|
| **supported_languages** | 27.35% | Keep as NaN, since language availability is an actual missing feature. |

- **Action: Keep supported_languages as NaN, since not all games support multiple languages. We don't want to introduce incorrect data.**

---

**Columns with High Missing Data (> 50%)**
| Column | Missing % | Suggested Action |
|---------|----------|-----------------|
| **achievement_description** | 69.11% | Fill missing values with "unknown". |
| **price** | 71.58% | Keep as NaN—missing prices could indicate unavailable data, regional limitations, or discontinued games. |

**Action:**
- **Fill achievement_description with "No description available"** to avoid empty fields.
- **Keep price as NaN**, since forcing imputation could lead to inaccurate pricing.

In [125]:
# fill missing values for categorical text columns
tidy_game_info["developers"].fillna("unknown", inplace=True)
tidy_game_info["publishers"].fillna("unknown", inplace=True)
tidy_game_info["genres"].fillna("unknown", inplace=True)
tidy_game_info["achievement_description"].fillna("unknown", inplace=True)

/var/folders/sd/5vy204s55_ggyjh65z1tlxg00000gn/T/ipykernel_43239/1875302043.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tidy_game_info["developers"].fillna("unknown", inplace=True)
/var/folders/sd/5vy204s55_ggyjh65z1tlxg00000gn/T/ipykernel_43239/1875302043.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

## 2. Multi-valued cols

In [126]:
tidy_game_info.sample(5)

,gameid,title,developers,publishers,genres,supported_languages,release_date,achievementid,achievement_title,achievement_description,platform,currency,price
3595266,2301100,Reverie: Sweet As Edition,['Rainbite Ltd'],['Eastasiasoft Limited'],"['Action', 'Adventure', 'Indie']","['English', 'French', 'German', 'Spanish - Spa...",2023-06-28,2301100_Snorkel,Ain't No River Wide Enough,Acquire the snorkel.,st,usd,6.49
12348987,2620210,Jinshin,['Exe Create Inc.'],['KEMCO'],"['Adventure', 'RPG', 'Simulation', 'Strategy']","['English', 'Japanese']",2023-12-21,2620210_5,Onigami Kegare,unknown,st,gbp,13.49
20530077,494730,Galacticare,['Brightrock Games'],['CULT Games'],"['Simulation', 'Strategy']","['English', 'French', 'German', 'Spanish - Spa...",2024-05-23,494730_bonus_mc2,Doc Idol,unknown,st,jpy,3110.00
10895556,220820,Zombie Driver HD,['Exor Studios'],['Exor Studios'],"['Action', 'Indie', 'Racing']","['English', 'French', 'German', 'Italian', 'Po...",2012-10-17,220820_kill_zombies_watercannon,It's not a weapon,Kill 100 zombies with a watercannon in one mis...,st,eur,1.99
25278535,444930,Zaccaria Pinball,['Magic Pixel Kft.'],['Magic Pixel Kft.'],"['Action', 'Casual', 'Free To Play', 'Indie', ...",['English'],2016-06-16,444930_ACH_A_SPEEDKING_CHALLENGE_SILVER,Speed King - Challenge Silver,Collect 6 points in Challenge game mode on Spe...,st,rub,NaN


In [127]:
tidy_game_info['title'].value_counts()

title
Zaccaria Pinball                          1084700
PAYDAY 2                                   299670
Assetto Corsa                              160775
The Binding of Isaac: Rebirth              146980
Idle Champions of the Forgotten Realms     141540
                                           ...   
Alice Sisters                                 405
Baldur's Gate: Dark Alliance                  375
Action SuperCross                             365
Star99                                        365
Skelattack                                    365
Name: count, Length: 3466, dtype: int64

#### Handling Multi-Valued Columns in Our Dataset [Used GPT-4o for tailoring grammar]

Our dataset contains multi-valued columns such as `developers`, `publishers`, `genres`, and `supported_languages`, which are stored as **list-like strings**. Instead of immediately expanding them into multiple rows (long format) or separate columns (one-hot encoding), we are **keeping them in their original format** for the following reasons:

1. **Data Integrity & Storage Efficiency**  
   - Expanding these fields would significantly increase row count, making storage and initial processing heavier.
   - Keeping them as lists allows us to preserve all information within a single row per game.

2. **Flexibility for Future Analysis**  
   - At later stages, we may **apply one-hot encoding** to `genres` and `supported_languages` for categorical analysis.  
   - Alternatively, we can derive **summary features** such as:
     - **Number of genres per game**
     - **Number of supported languages**
     - **Unique count of developers/publishers**
   - These derived features will allow for a more structured comparison across games.

By postponing transformation, we maintain efficiency while keeping the option open for structured feature extraction when needed.


In [128]:
# tidy_game_info_noids = tidy_game_info.drop(columns=['gameid', 'achievementid'], inplace=True)

## More notes on further merging

Our current **tidy dataset** focuses solely on game-related information. To enrich the analysis, we plan additional merges with player data to uncover insights about game popularity, completion rates, and player demographics.

### Planned Merges & Insights
1. **Game Popularity Analysis**  
   - Merge with `purchased_games.csv` (for each platform) to **count the number of players who own each game**.
   - This helps identify **best-selling games** and platform-specific purchase trends.

2. **Game Completion Rates**  
   - Merge with `history.csv` to **track when players earn end-game achievements**.
   - Calculate the **average time to completion** and **percentage of players finishing a game**.
   - Investigate if certain **genres have higher completion rates** (e.g., RPGs vs. casual games).
  
3. **Pricing & Sales Relationship:**  
   - Merge `prices.csv` to analyze how **price fluctuations impact purchases**.
  
4. **Cross-Platform Player Behavior:**  
   - Identify players who own **the same game across multiple platforms** to study cross-platform engagement.
   - Merge with `players.csv` and **cluster by country** to analyze **regional gaming preferences**.


## [TODO] Data Visualization
> Be sure to include interpretations of your visualizations -- what patterns or anomalies do you see?


###  Player Engagement Visualization: player-game-achievement

In [129]:
# Top 50 most popular games
# active players
# Top 50 most popular achievements


player_game_achievement = 
# how many achievements a player unlocked per game

# Group data to count achievements per player per game
agg_data = ps_player_info.groupby(["playerid", "gameid"])["achievementid"].count().reset_index()
agg_data.rename(columns={"achievementid": "achievements_unlocked"}, inplace=True)

# Sample a subset (e.g., 10K rows) for visualization
sample_data = agg_data.sample(10000, random_state=42)

SyntaxError: invalid syntax (3815978410.py, line 6)